In [1]:
import os
import json
import math
import pandas as pd
import string
from pyvi.ViTokenizer import tokenize
import pandas as pd
import numpy as np
import json, pickle
from rank_bm25 import BM25Okapi
import argparse
import gc
from tqdm.auto import tqdm
tqdm.pandas()
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import string
import ast

from pandarallel import pandarallel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
def get_topk(query, topk = 100):
    tokenized_query = query.split()
    tfidf_query = tfidf_model[dictionary.doc2bow(tokenized_query)]
    scores = bm25_index[tfidf_query]
    top_n = np.argsort(scores)[::-1][:topk]
    titles = [df_wiki.title.values[i] for i in top_n]
    texts = [df_wiki.text.values[i] for i in top_n]
    # print(titles)
    # print(tfidf_query, scores)
    return titles, texts, scores[top_n]

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

In [3]:
legal_cleaned_path =  "../processed_data/legal_df_embedding_numpy.csv"
topk = 300

In [4]:
df_wiki = pd.read_csv(legal_cleaned_path)
df_wiki['embedding'] = df_wiki['embedding'].apply(ast.literal_eval)
df_wiki['embedding'] = df_wiki['embedding'].apply(lambda x: np.array([x]))

In [5]:
print(df_wiki.iloc[0]['bm25_text'].lower())
print(df_wiki.iloc[0]['segment'])

phạm vi điều chỉnh luật này quy định về viên chức quyền nghĩa vụ của viên chức tuyển dụng sử dụng và quản lý viên chức trong đơn vị sự nghiệp công lập
Phạm_vi điều_chỉnh Luật này quy_định về viên_chức ; quyền nghĩa_vụ của viên_chức ; tuyển_dụng , sử_dụng và quản_lý_viên_chức trong đơn_vị sự_nghiệp công_lập .


In [7]:
df_wiki.dropna(inplace=True, ignore_index=True)

In [31]:
df_wiki['bm25_text'] = df_wiki['bm25_text'].apply(lambda t: t.lower())
corpus = [x.split() for x in df_wiki['bm25_text'].values]
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save("../bm25/bm25_stage1/dict")
tfidf_model.save("../bm25/bm25_stage1/tfidf")
bm25_index.save("../bm25/bm25_stage1/bm25_index")

In [8]:
class BM25Gensim:
    def __init__(self, checkpoint_path):
        self.dictionary = Dictionary.load(checkpoint_path + "/dict")
        self.tfidf_model = SparseMatrixSimilarity.load(checkpoint_path + "/tfidf")
        self.bm25_index = TfidfModel.load(checkpoint_path + "/bm25_index")

    def get_topk_stage1(self, query, topk=100):
        tokenized_query = query.split()
        tfidf_query = self.tfidf_model[self.dictionary.doc2bow(tokenized_query)]
        scores = self.bm25_index[tfidf_query]
        top_n = np.argsort(scores)[::-1][:topk]
        return top_n, scores[top_n]

In [9]:
bm25_model_stage1 = BM25Gensim("../bm25/bm25_stage1/")

In [10]:
train_df = pd.read_csv('../processed_data/train_df.csv')
train_df

,example_id,label,statement,law_id,article_id,segment
0,q9zjh7Uw7Q,No,Người xem dưới 16 tuổi được xem phim có nội du...,Luật Điện ảnh 2022,32,Người xem dưới 16 tuổi được xem phim có nội_du...
1,ckQFn8y202,No,"Trong vòng 03 ngày làm việc, kể từ ngày người ...","Luật Phòng, chống ma túy 2021",30,"Trong vòng 03 ngày làm_việc , kể từ ngày người..."
2,3ROu621ZEO,Yes,Viên chức có 02 năm liên tiếp bị phân loại đán...,Luật Viên chức 2010,29,Viên_chức có 02 năm liên_tiếp bị phân_loại đán...
3,VT1QuVmhCc,Yes,Các biện pháp cai nghiện ma túy là những biện ...,"Luật Phòng, chống ma túy 2021",28,Các biện_pháp cai_nghiện ma_tuý là những biện_...
4,0MwITLtbmg,No,Viên chức thuộc một đơn vị sự nghiệp công lập ...,Luật Viên chức 2010,14,Viên_chức thuộc một đơn_vị sự_nghiệp công_lập ...
...,...,...,...,...,...,...
71,3lEnngVd8Z,Yes,Viên chức bị khiển trách thì thời hạn nâng lươ...,Luật Viên chức 2010,9,Viên_chức bị khiển_trách thì thời_hạn nâng lươ...
72,6nfjyV5thx,No,"Cơ quan nhà nước không có thẩm quyền theo dõi,...","Luật Phòng, chống ma túy 2021",14,Cơ_quan nhà_nước không có thẩm_quyền theo_dõi ...
73,TisBomZhjP,Yes,Cách chức là một trong các hình thức xử lý kỷ ...,Luật Viên chức 2010,52,Cách_chức là một trong các hình_thức xử_lý kỷ_...
74,O8JI0xlIS5,Yes,Hợp đồng làm việc đối với trường hợp người đượ...,Luật Viên chức 2010,26,Hợp_đồng làm_việc đối_với trường_hợp người đượ...


In [11]:
import json
from glob import glob
import re
from nltk import word_tokenize as lib_tokenizer
import string


def preprocess(x, max_length=-1, remove_puncts=False):
    x = nltk_tokenize(x)
    x = x.replace("\n", " ")
    if remove_puncts:
        x = "".join([i for i in x if i not in string.punctuation])
    if max_length > 0:
        x = " ".join(x.split()[:max_length])
    return x

def nltk_tokenize(x):
    return " ".join(word_tokenize(strip_context(x))).strip()

In [12]:
idx = 2
print(train_df.iloc[idx])
query = preprocess(train_df.iloc[idx]['statement']).lower()
query

example_id                                           3ROu621ZEO
label                                                       Yes
statement     Viên chức có 02 năm liên tiếp bị phân loại đán...
law_id                                      Luật Viên chức 2010
article_id                                                   29
segment       Viên_chức có 02 năm liên_tiếp bị phân_loại đán...
Name: 2, dtype: object


'viên chức có 02 năm liên tiếp bị phân loại đánh giá ở mức độ không hoàn thành nhiệm vụ'

In [13]:
top_n, bm25_scores = bm25_model_stage1.get_topk_stage1(query, 10)

In [14]:
res = df_wiki.loc[top_n]
res

,law_id,article_id,text,segment,embedding,bm25_text
61,Luật Viên chức 2010,42,"Phân loại đánh giá viên chức\n\n1. Hàng năm, c...","Phân_loại đánh_giá_viên_chức 1 . Hàng năm , că...","[[0.2947031855583191, 0.23052316904067993, -0....",phân loại đánh giá viên chức 1 hàng năm căn cứ...
38,Luật Viên chức 2010,29,Đơn phương chấm dứt hợp đồng làm việc\n\n1. Đơ...,Đơn_phương chấm_dứt hợp_đồng làm_việc 1 . Đơn_...,"[[0.12286890298128128, -0.011710107326507568, ...",đơn phương chấm dứt hợp đồng làm việc 1 đơn vị...
63,Luật Viên chức 2010,44,"Thông báo kết quả đánh giá, phân loại viên chứ...","Thông_báo kết_quả đánh_giá , phân_loại_viên_ch...","[[0.5046664476394653, -0.04340879246592522, 0....",thông báo kết quả đánh giá phân loại viên chức...
62,Luật Viên chức 2010,43,Trách nhiệm đánh giá viên chức\n\n1. Người đứn...,Trách_nhiệm đánh_giá_viên_chức 1 . Người đứng ...,"[[0.12494437396526337, 0.11893761157989502, 0....",trách nhiệm đánh giá viên chức 1 người đứng đầ...
57,Luật Viên chức 2010,39,Mục đích của đánh giá viên chức\n\nMục đích củ...,Mục_đích của đánh_giá_viên_chức Mục_đích của đ...,"[[0.28394365310668945, 0.23940877616405487, -0...",mục đích của đánh giá viên chức mục đích của đ...
77,Luật Viên chức 2010,55,"Trách nhiệm bồi thường, hoàn trả\n\n1. Viên ch...","Trách_nhiệm bồi_thường , hoàn_trả 1 . Viên_chứ...","[[0.32398203015327454, -0.26925647258758545, 0...",trách nhiệm bồi thường hoàn trả 1 viên chức là...
59,Luật Viên chức 2010,41,Nội dung đánh giá viên chức\n\n1. Việc đánh gi...,Nội_dung đánh_giá_viên_chức 1 . Việc đánh_giá_...,"[[0.21189919114112854, 0.18901577591896057, 0....",nội dung đánh giá viên chức 1 việc đánh giá vi...
1521,Luật Thanh tra 2022,42,Miễn nhiệm Thanh tra viên\n\n1. Việc miễn nhiệ...,Miễn_nhiệm Thanh_tra_viên 1 . Việc miễn_nhiệm ...,"[[0.021657578647136688, 0.0376962348818779, -0...",miễn nhiệm thanh tra viên 1 việc miễn nhiệm th...
1543,Luật Thanh tra 2022,54,Xử lý hành vi vi phạm pháp luật của người tiến...,Xử_lý hành_vi vi_phạm pháp_luật của người tiến...,"[[-0.12053010612726212, 0.07890800386667252, -...",xử lý hành vi vi phạm pháp luật của người tiến...
2542,Luật Tố tụng hành chính 2015,91,"Định giá tài sản, thẩm định giá tài sản\n\n1. ...",tổ_chức thẩm_định giá để thẩm_định giá tài_sản...,"[[-0.1699284166097641, -0.3105523884296417, 0....",tổ chức thẩm định giá để thẩm định giá tài sản...


In [15]:
bm25_scores = bm25_scores
bm25_scores

array([41.820065, 38.4315  , 29.859472, 24.680992, 23.688866, 22.482533,
       22.24757 , 21.515476, 21.416851, 21.26966 ], dtype=float32)

In [16]:
res['bm25_score'] = bm25_scores
res

,law_id,article_id,text,segment,embedding,bm25_text,bm25_score
61,Luật Viên chức 2010,42,"Phân loại đánh giá viên chức\n\n1. Hàng năm, c...","Phân_loại đánh_giá_viên_chức 1 . Hàng năm , că...","[[0.2947031855583191, 0.23052316904067993, -0....",phân loại đánh giá viên chức 1 hàng năm căn cứ...,41.820065
38,Luật Viên chức 2010,29,Đơn phương chấm dứt hợp đồng làm việc\n\n1. Đơ...,Đơn_phương chấm_dứt hợp_đồng làm_việc 1 . Đơn_...,"[[0.12286890298128128, -0.011710107326507568, ...",đơn phương chấm dứt hợp đồng làm việc 1 đơn vị...,38.431499
63,Luật Viên chức 2010,44,"Thông báo kết quả đánh giá, phân loại viên chứ...","Thông_báo kết_quả đánh_giá , phân_loại_viên_ch...","[[0.5046664476394653, -0.04340879246592522, 0....",thông báo kết quả đánh giá phân loại viên chức...,29.859472
62,Luật Viên chức 2010,43,Trách nhiệm đánh giá viên chức\n\n1. Người đứn...,Trách_nhiệm đánh_giá_viên_chức 1 . Người đứng ...,"[[0.12494437396526337, 0.11893761157989502, 0....",trách nhiệm đánh giá viên chức 1 người đứng đầ...,24.680992
57,Luật Viên chức 2010,39,Mục đích của đánh giá viên chức\n\nMục đích củ...,Mục_đích của đánh_giá_viên_chức Mục_đích của đ...,"[[0.28394365310668945, 0.23940877616405487, -0...",mục đích của đánh giá viên chức mục đích của đ...,23.688866
77,Luật Viên chức 2010,55,"Trách nhiệm bồi thường, hoàn trả\n\n1. Viên ch...","Trách_nhiệm bồi_thường , hoàn_trả 1 . Viên_chứ...","[[0.32398203015327454, -0.26925647258758545, 0...",trách nhiệm bồi thường hoàn trả 1 viên chức là...,22.482533
59,Luật Viên chức 2010,41,Nội dung đánh giá viên chức\n\n1. Việc đánh gi...,Nội_dung đánh_giá_viên_chức 1 . Việc đánh_giá_...,"[[0.21189919114112854, 0.18901577591896057, 0....",nội dung đánh giá viên chức 1 việc đánh giá vi...,22.247570
1521,Luật Thanh tra 2022,42,Miễn nhiệm Thanh tra viên\n\n1. Việc miễn nhiệ...,Miễn_nhiệm Thanh_tra_viên 1 . Việc miễn_nhiệm ...,"[[0.021657578647136688, 0.0376962348818779, -0...",miễn nhiệm thanh tra viên 1 việc miễn nhiệm th...,21.515476
1543,Luật Thanh tra 2022,54,Xử lý hành vi vi phạm pháp luật của người tiến...,Xử_lý hành_vi vi_phạm pháp_luật của người tiến...,"[[-0.12053010612726212, 0.07890800386667252, -...",xử lý hành vi vi phạm pháp luật của người tiến...,21.416851
2542,Luật Tố tụng hành chính 2015,91,"Định giá tài sản, thẩm định giá tài sản\n\n1. ...",tổ_chức thẩm_định giá để thẩm_định giá tài_sản...,"[[-0.1699284166097641, -0.3105523884296417, 0....",tổ chức thẩm định giá để thẩm định giá tài sản...,21.269661


In [17]:
import sys

In [18]:
sys.path.append('../')
print(sys.path)

['/storage/nghiatl/Legal_VLSP/notebooks', '/home/nghiatl/anaconda3/envs/chatgpt/lib/python38.zip', '/home/nghiatl/anaconda3/envs/chatgpt/lib/python3.8', '/home/nghiatl/anaconda3/envs/chatgpt/lib/python3.8/lib-dynload', '', '/home/nghiatl/anaconda3/envs/chatgpt/lib/python3.8/site-packages', '../']


In [19]:
from retrieval_model import Retrieval_model
from text_utils import segment_text

Vncorenlp loaded


In [52]:
encoder_model = Retrieval_model()

In [55]:
res2 = encoder_model.search_related(df_wiki, train_df.iloc[idx]['segment'], 10, False)
res2

torch.Size([1, 768])


,law_id,article_id,text,segment,embedding,bm25_text,similarity
61,Luật Viên chức 2010,42,"Phân loại đánh giá viên chức\n\n1. Hàng năm, c...","Phân_loại đánh_giá_viên_chức 1 . Hàng năm , că...","[[0.2947031855583191, 0.23052316904067993, -0....",phân loại đánh giá viên chức 1 hàng năm căn cứ...,[[0.577908431861081]]
78,Luật Viên chức 2010,56,Các quy định khác liên quan đến việc kỷ luật v...,Các quy_định khác liên_quan đến việc kỷ_luật_v...,"[[0.21398159861564636, 0.11541047692298889, -0...",các quy định khác liên quan đến việc kỷ luật v...,[[0.5099043863995257]]
1518,Luật Thanh tra 2022,39,Tiêu chuẩn bổ nhiệm vào ngạch thanh tra viên\n...,"về cán_bộ , công_chức , viên_chức . 5 . Có ít_...","[[0.2524369955062866, -0.22180381417274475, 0....",về cán bộ công chức viên chức 5 có ít nhất 02 ...,[[0.49105554179492295]]
79,Luật Viên chức 2010,56,Các quy định khác liên quan đến việc kỷ luật v...,hoặc bị Toà_án kết_án về hành_vi tham_nhũng th...,"[[0.11672565340995789, -0.045936889946460724, ...",hoặc bị toà án kết án về hành vi tham nhũng th...,[[0.4879103917853579]]
80,Luật Viên chức 2010,57,Quy định đối với viên chức bị truy cứu trách n...,Quy_định đối_với_viên_chức bị truy_cứu trách_n...,"[[0.2080373466014862, -0.12226226180791855, -0...",quy định đối với viên chức bị truy cứu trách n...,[[0.47915514226702793]]
38,Luật Viên chức 2010,29,Đơn phương chấm dứt hợp đồng làm việc\n\n1. Đơ...,Đơn_phương chấm_dứt hợp_đồng làm_việc 1 . Đơn_...,"[[0.12286890298128128, -0.011710107326507568, ...",đơn phương chấm dứt hợp đồng làm việc 1 đơn vị...,[[0.4739252763729355]]
74,Luật Viên chức 2010,53,"Thời hiệu, thời hạn xử lý kỷ luật\n\n1. Thời h...","Thời_hiệu , thời_hạn xử_lý kỷ_luật 1 . Thời_hi...","[[0.20693281292915344, -0.012596381828188896, ...",thời hiệu thời hạn xử lý kỷ luật 1 thời hiệu x...,[[0.46381727720712307]]
76,Luật Viên chức 2010,54,Tạm đình chỉ công tác\n\n1. Trong thời hạn xử ...,Tạm đình_chỉ công_tác 1 . Trong thời_hạn xử_lý...,"[[0.27406060695648193, 0.016122611239552498, 0...",tạm đình chỉ công tác 1 trong thời hạn xử lý k...,[[0.45845500905875036]]
52,Luật Viên chức 2010,36,Biệt phái viên chức\n\n1. Biệt phái viên chức ...,", vùng dân_tộc_thiểu_số , vùng có điều_kiện ki...","[[0.137412428855896, 0.05972390994429588, 0.16...",vùng dân tộc thiểu số vùng có điều kiện kinh t...,[[0.45465135745542823]]
23,Luật Viên chức 2010,19,Những việc viên chức không được làm\n\n1. Trốn...,Những việc viên_chức không được làm 1 . Trốn_t...,"[[0.10267068445682526, -0.055069852620363235, ...",những việc viên chức không được làm 1 trốn trá...,[[0.4518241817656041]]


In [27]:
def agrr_score(row):
    if np.isnan(row['similarity']):
        row['total_score'] = row['bm25_score'] + 0.001
        row['similarity'] = 0.001
        return row
    if np.isnan(row['bm25_score']):
        row['similarity'] = row['similarity'][0][0]
        row['total_score'] = row['similarity'] + 0.001
        row['bm25_score'] = 0.001
        return row
    row['similarity'] = row['similarity'][0][0]
    row['total_score'] = row['bm25_score'] + row['similarity']
    return row

In [60]:
df_wiki.drop(columns=['similarity'], inplace=True)

In [61]:
df_wiki

,law_id,article_id,text,segment,embedding,bm25_text
0,Luật Viên chức 2010,1,Phạm vi điều chỉnh\n\nLuật này quy định về viê...,Phạm_vi điều_chỉnh Luật này quy_định về viên_c...,"[[0.16507713496685028, 0.08745536208152771, 0....",phạm vi điều chỉnh luật này quy định về viên c...
1,Luật Viên chức 2010,2,Viên chức\n\nViên chức là công dân Việt Nam đư...,Viên_chức_Viên_chức là công_dân Việt_Nam được ...,"[[0.29940927028656006, 0.03183652088046074, -0...",viên chức viên chức là công dân việt nam được ...
2,Luật Viên chức 2010,3,"Giải thích từ ngữ\n\nTrong Luật này, các từ ng...","Giải_thích từ_ngữ Trong Luật này , các từ_ngữ ...","[[0.1526578664779663, -0.04446101933717728, -0...",giải thích từ ngữ trong luật này các từ ngữ dư...
3,Luật Viên chức 2010,3,"Giải thích từ ngữ\n\nTrong Luật này, các từ ng...","có phẩm_chất , trình_độ và năng_lực vào làm vi...","[[0.14661511778831482, -0.006432163529098034, ...",có phẩm chất trình độ và năng lực vào làm viên...
4,Luật Viên chức 2010,4,Hoạt động nghề nghiệp của viên chức\n\nHoạt độ...,Hoạt_động nghề_nghiệp của viên_chức Hoạt_động ...,"[[0.14538682997226715, -0.12979193031787872, 0...",hoạt động nghề nghiệp của viên chức hoạt động ...
...,...,...,...,...,...,...
3468,Luật Thanh niên 2020,39,"Trách nhiệm của các Bộ, cơ quan ngang Bộ\n\nCá...","pháp_luật đối_với thanh_niên theo ngành , lĩnh...","[[0.3031163513660431, -0.2225814312696457, 0.3...",pháp luật đối với thanh niên theo ngành lĩnh v...
3469,Luật Thanh niên 2020,40,"Trách nhiệm của Hội đồng nhân dân, Ủy ban nhân...","Trách_nhiệm của Hội_đồng_nhân_dân , Uỷ_ban_nhâ...","[[-0.26759663224220276, 0.055772919207811356, ...",trách nhiệm của hội đồng nhân dân uỷ ban nhân ...
3470,Luật Thanh niên 2020,40,"Trách nhiệm của Hội đồng nhân dân, Ủy ban nhân...","mình , thực_hiện quản_lý_nhà_nước về thanh_niê...","[[0.08777713030576706, -0.07565575838088989, 0...",mình thực hiện quản lý nhà nước về thanh niên ...
3471,Luật Thanh niên 2020,40,"Trách nhiệm của Hội đồng nhân dân, Ủy ban nhân...","thực_hiện chính_sách , pháp_luật về thanh_niên...","[[0.12399047613143921, -0.05082567781209946, 0...",thực hiện chính sách pháp luật về thanh niên g...


In [28]:
merged_df = pd.merge(res, res2, on=['law_id', 'article_id', 'segment','text', 'bm25_text'], suffixes=('_df1', '_df2'), how='outer')
merged_df = merged_df.apply(agrr_score, axis=1)
merged_df

,law_id,article_id,text,segment,embedding_df1,bm25_text,bm25_score,embedding_df2,similarity,total_score
0,Luật Viên chức 2010,42,"Phân loại đánh giá viên chức\n\n1. Hàng năm, c...","Phân_loại đánh_giá_viên_chức 1 . Hàng năm , că...","[[0.2947031855583191, 0.23052316904067993, -0....",phân loại đánh giá viên chức 1 hàng năm căn cứ...,41.820065,"[[0.2947031855583191, 0.23052316904067993, -0....",0.577908,42.397973
1,Luật Viên chức 2010,29,Đơn phương chấm dứt hợp đồng làm việc\n\n1. Đơ...,Đơn_phương chấm_dứt hợp_đồng làm_việc 1 . Đơn_...,"[[0.12286890298128128, -0.011710107326507568, ...",đơn phương chấm dứt hợp đồng làm việc 1 đơn vị...,38.431499,"[[0.12286890298128128, -0.011710107326507568, ...",0.473925,38.905425
2,Luật Viên chức 2010,44,"Thông báo kết quả đánh giá, phân loại viên chứ...","Thông_báo kết_quả đánh_giá , phân_loại_viên_ch...","[[0.5046664476394653, -0.04340879246592522, 0....",thông báo kết quả đánh giá phân loại viên chức...,29.859472,NaN,0.001000,29.860472
3,Luật Viên chức 2010,43,Trách nhiệm đánh giá viên chức\n\n1. Người đứn...,Trách_nhiệm đánh_giá_viên_chức 1 . Người đứng ...,"[[0.12494437396526337, 0.11893761157989502, 0....",trách nhiệm đánh giá viên chức 1 người đứng đầ...,24.680992,NaN,0.001000,24.681992
4,Luật Viên chức 2010,39,Mục đích của đánh giá viên chức\n\nMục đích củ...,Mục_đích của đánh_giá_viên_chức Mục_đích của đ...,"[[0.28394365310668945, 0.23940877616405487, -0...",mục đích của đánh giá viên chức mục đích của đ...,23.688866,NaN,0.001000,23.689866
5,Luật Viên chức 2010,55,"Trách nhiệm bồi thường, hoàn trả\n\n1. Viên ch...","Trách_nhiệm bồi_thường , hoàn_trả 1 . Viên_chứ...","[[0.32398203015327454, -0.26925647258758545, 0...",trách nhiệm bồi thường hoàn trả 1 viên chức là...,22.482533,NaN,0.001000,22.483533
6,Luật Viên chức 2010,41,Nội dung đánh giá viên chức\n\n1. Việc đánh gi...,Nội_dung đánh_giá_viên_chức 1 . Việc đánh_giá_...,"[[0.21189919114112854, 0.18901577591896057, 0....",nội dung đánh giá viên chức 1 việc đánh giá vi...,22.247570,NaN,0.001000,22.248570
7,Luật Thanh tra 2022,42,Miễn nhiệm Thanh tra viên\n\n1. Việc miễn nhiệ...,Miễn_nhiệm Thanh_tra_viên 1 . Việc miễn_nhiệm ...,"[[0.021657578647136688, 0.0376962348818779, -0...",miễn nhiệm thanh tra viên 1 việc miễn nhiệm th...,21.515476,NaN,0.001000,21.516476
8,Luật Thanh tra 2022,54,Xử lý hành vi vi phạm pháp luật của người tiến...,Xử_lý hành_vi vi_phạm pháp_luật của người tiến...,"[[-0.12053010612726212, 0.07890800386667252, -...",xử lý hành vi vi phạm pháp luật của người tiến...,21.416851,NaN,0.001000,21.417851
9,Luật Tố tụng hành chính 2015,91,"Định giá tài sản, thẩm định giá tài sản\n\n1. ...",tổ_chức thẩm_định giá để thẩm_định giá tài_sản...,"[[-0.1699284166097641, -0.3105523884296417, 0....",tổ chức thẩm định giá để thẩm định giá tài sản...,21.269661,NaN,0.001000,21.270661


In [42]:
segment_text(train_df.iloc[idx]['statement'])

'Viên_chức có 02 năm liên_tiếp bị phân_loại đánh_giá ở mức_độ không hoàn_thành nhiệm_vụ'

In [70]:
def retrieval_stage(statement, legal_df, bm25_model_stage1, encoder_model):
    query = preprocess(statement).lower()
    top_n, bm25_scores = bm25_model_stage1.get_topk_stage1(query, 10)
    res = legal_df.loc[top_n]
    res['bm25_score'] = bm25_scores
    query = segment_text(statement)
    res2 = encoder_model.search_related(legal_df.copy(), query, 10, False)
    merged_df = pd.merge(res, res2, on=['law_id', 'article_id', 'segment','text', 'bm25_text'], suffixes=('_df1', '_df2'), how='outer')
    merged_df = merged_df.apply(agrr_score, axis=1)
    results = (
            merged_df.sort_values("total_score", ascending=False).head(5)
    )
    
    return results

In [62]:
retrieval_stage(train_df.iloc[idx]['statement'], df_wiki, bm25_model_stage1, encoder_model)

torch.Size([1, 768])


,law_id,article_id,text,segment,embedding_df1,bm25_text,bm25_score,embedding_df2,similarity,total_score
0,Luật Viên chức 2010,42,"Phân loại đánh giá viên chức\n\n1. Hàng năm, c...","Phân_loại đánh_giá_viên_chức 1 . Hàng năm , că...","[[0.2947031855583191, 0.23052316904067993, -0....",phân loại đánh giá viên chức 1 hàng năm căn cứ...,41.820065,"[[0.2947031855583191, 0.23052316904067993, -0....",0.577908,42.397973
1,Luật Viên chức 2010,29,Đơn phương chấm dứt hợp đồng làm việc\n\n1. Đơ...,Đơn_phương chấm_dứt hợp_đồng làm_việc 1 . Đơn_...,"[[0.12286890298128128, -0.011710107326507568, ...",đơn phương chấm dứt hợp đồng làm việc 1 đơn vị...,38.431499,"[[0.12286890298128128, -0.011710107326507568, ...",0.473925,38.905425
2,Luật Viên chức 2010,44,"Thông báo kết quả đánh giá, phân loại viên chứ...","Thông_báo kết_quả đánh_giá , phân_loại_viên_ch...","[[0.5046664476394653, -0.04340879246592522, 0....",thông báo kết quả đánh giá phân loại viên chức...,29.859472,NaN,0.001000,29.860472
3,Luật Viên chức 2010,43,Trách nhiệm đánh giá viên chức\n\n1. Người đứn...,Trách_nhiệm đánh_giá_viên_chức 1 . Người đứng ...,"[[0.12494437396526337, 0.11893761157989502, 0....",trách nhiệm đánh giá viên chức 1 người đứng đầ...,24.680992,NaN,0.001000,24.681992
4,Luật Viên chức 2010,39,Mục đích của đánh giá viên chức\n\nMục đích củ...,Mục_đích của đánh_giá_viên_chức Mục_đích của đ...,"[[0.28394365310668945, 0.23940877616405487, -0...",mục đích của đánh giá viên chức mục đích của đ...,23.688866,NaN,0.001000,23.689866
5,Luật Viên chức 2010,55,"Trách nhiệm bồi thường, hoàn trả\n\n1. Viên ch...","Trách_nhiệm bồi_thường , hoàn_trả 1 . Viên_chứ...","[[0.32398203015327454, -0.26925647258758545, 0...",trách nhiệm bồi thường hoàn trả 1 viên chức là...,22.482533,NaN,0.001000,22.483533
6,Luật Viên chức 2010,41,Nội dung đánh giá viên chức\n\n1. Việc đánh gi...,Nội_dung đánh_giá_viên_chức 1 . Việc đánh_giá_...,"[[0.21189919114112854, 0.18901577591896057, 0....",nội dung đánh giá viên chức 1 việc đánh giá vi...,22.247570,NaN,0.001000,22.248570
7,Luật Thanh tra 2022,42,Miễn nhiệm Thanh tra viên\n\n1. Việc miễn nhiệ...,Miễn_nhiệm Thanh_tra_viên 1 . Việc miễn_nhiệm ...,"[[0.021657578647136688, 0.0376962348818779, -0...",miễn nhiệm thanh tra viên 1 việc miễn nhiệm th...,21.515476,NaN,0.001000,21.516476
8,Luật Thanh tra 2022,54,Xử lý hành vi vi phạm pháp luật của người tiến...,Xử_lý hành_vi vi_phạm pháp_luật của người tiến...,"[[-0.12053010612726212, 0.07890800386667252, -...",xử lý hành vi vi phạm pháp luật của người tiến...,21.416851,NaN,0.001000,21.417851
9,Luật Tố tụng hành chính 2015,91,"Định giá tài sản, thẩm định giá tài sản\n\n1. ...",tổ_chức thẩm_định giá để thẩm_định giá tài_sản...,"[[-0.1699284166097641, -0.3105523884296417, 0....",tổ chức thẩm định giá để thẩm định giá tài sản...,21.269661,NaN,0.001000,21.270661


In [71]:
test_df = pd.read_csv('../processed_data/test_df.csv')
test_df

,example_id,statement,law_id,article_id
0,qr6S2jA9GG,"Nếu không phạm tội quả tang, một người sẽ khôn...",Hiến pháp 2013,20
1,zFWTqve74q,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7
2,bv98C2F6I9,Hoạt động thanh tra phải được thực hiện theo k...,Luật Thanh tra 2022,46
3,Kzmq9qdScq,Loại hợp đồng nào sau đây không được BLDS 2015...,Bộ Luật Dân sự 2015,402
4,8x8IdMONw9,Bản án phúc thẩm được tuyên vào ngày 20/01/202...,Luật Tố tụng hành chính 2015,242
...,...,...,...,...
135,b5oiFGleY3,"Tháng hành động quốc gia phòng, chống bạo lực ...","Luật Phòng, chống bạo lực gia đình 2022",7
136,BHc3XXLIVa,"Theo Luật Điện ảnh, khi đi xem phim loại C tại...",Luật Điện ảnh 2022,32
137,cnBEeaS6WU,Cơ quan điều tra Viện kiểm sát nhân dân tối ca...,Luật Tổ chức viện kiểm sát nhân dân 2014,20
138,2Ta72q8BEz,Điều nào không là quyền hiến định của công dân...,Hiến pháp 2013,19


In [75]:
def retrieval_eval(df):
    true_pre = 0
    for idx in range(len(df)):
        statement = df.iloc[idx]['statement']
        law_id = df.iloc[idx]['law_id']
        article_id = df.iloc[idx]['article_id']
        rank_df = retrieval_stage(statement, df_wiki, bm25_model_stage1, encoder_model)
        if rank_df.iloc[0]['law_id'] == law_id and rank_df.iloc[0]['article_id'] == article_id:
            true_pre += 1
    print(len(df))
    print(true_pre)
    return true_pre/ len(df)

In [76]:
retrieval_eval(test_df)

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1

0.8